Activate Python virtual environment first and install all packages before running the notebook.

1. create a new virtual environment
```
python -m venv venv
```

2. activate the virtual environment
```
source venv/Scripts/activate
```

3. install packages
```
pip install -r requirements.txt
```

4. make sure your kernel is switched to the venv python kernel






Goal: Examine correlations between global,socially significant events and Bitcoin blockchain metrics (block congestion, price movement, metadata notes) over time.
Utilizing publick bitcoin blockchain datasets stored in bigquery
to begin, I installed the following libraries:
pip install google-cloud-bigquery pandas pyarrow

Once I started trying to communicate with bigquery database, I had to also install:
pip install db-dtypes
to help pandas understand how to display data types used in SQL like "DATE, TIMESTAMP, STRUCT,etc.

Tables used:
 
A. bigquery-public-data.crypto_bitcoin.blocks table
B. bigquery-public-data.crypto_bitcoin.transactions table



Note to myself, I need to send a slack message to Eileen with the content of my credentials file and tell her she needs this to run my project.

In [2]:
import sys
print(sys.executable)


c:\C drive Code You Projects\Code You Data Analyst Course Module 3\Module 3 Capstone Project\venv\Scripts\python.exe


In [3]:
from google.cloud import bigquery
import os
#Set the environement variable for key file (telling python where my login key file is)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Credentials/config.json"
#Start the BigQuery client using my key
client = bigquery.Client()



Due to data constraints, I decided to refine my dataset to focus on the 1 month period before/after covid lockdown took place in the US (March 1st 2020~March 30th 2020)
Due to issues to the original datasets I was working with (not having data for the date ranges I wanted), I need to change and update the list of tables I'm using



In [4]:
# Checking if the new table "crypto_bitcoin.blocks" has information for the timeperiod I'm looking for:
query_sample = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.blocks`
LIMIT 5
"""

df_sample = client.query(query_sample).to_dataframe()
df_sample.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   hash               5 non-null      object             
 1   size               5 non-null      Int64              
 2   stripped_size      5 non-null      Int64              
 3   weight             5 non-null      Int64              
 4   number             5 non-null      Int64              
 5   version            5 non-null      Int64              
 6   merkle_root        5 non-null      object             
 7   timestamp          5 non-null      datetime64[us, UTC]
 8   timestamp_month    5 non-null      dbdate             
 9   nonce              5 non-null      object             
 10  bits               5 non-null      object             
 11  coinbase_param     5 non-null      object             
 12  transaction_count  5 non-null      Int64              

In [5]:
# Checking if the new table "crypto_bitcoin.blocks" has information for the timeperiod I'm looking for:
query_check_range = """
SELECT
  MIN(timestamp) AS start_date,
  MAX(timestamp) AS end_date
FROM `bigquery-public-data.crypto_bitcoin.blocks`
"""

df_range = client.query(query_check_range).to_dataframe()
df_range

,start_date,end_date
0,2009-01-03 18:15:05+00:00,2025-08-03 13:48:08+00:00


so we made sure the information in the crypto_bitcoin.blocks table has block information starting 2009~today.
now checking the type of information obtained on the crypto_bitcoin.transactions table:

In [6]:
# Now checking to make sure there's enough data during Jan-June 2020 in the transactions table:
query_row_count = """
SELECT COUNT(*) as transaction_count
FROM `bigquery-public-data.crypto_bitcoin.transactions`
WHERE block_timestamp BETWEEN '2020-01-01' AND '2020-06-30'
"""
df_row_count = client.query(query_row_count).to_dataframe()
print(df_row_count)

   transaction_count
0           54643498


Since I found out the bitcoin transactions table for Jan~June of 2020 goes up to 54 Million rows, I tried narrowing my dataset to March of 2020, which was the most significant month for financial and social impact. After checking the number of rows of transactions during March 2020 adds up to 10 Million rows, I decided to extract a sample of 100K rows of data.

In [17]:
# Getting a sample (100k rows) of data from the bitcoin transactions table for March 2020:
query_transactions_march_2020 = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.transactions`
WHERE block_timestamp BETWEEN '2020-03-01' AND '2020-03-31 23:59:59'
LIMIT 100000
"""
df_transactions = client.query(query_transactions_march_2020).to_dataframe()
df_transactions.info()
df_transactions.head(45)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   hash                   100000 non-null  object             
 1   size                   100000 non-null  Int64              
 2   virtual_size           100000 non-null  Int64              
 3   version                100000 non-null  Int64              
 4   lock_time              100000 non-null  Int64              
 5   block_hash             100000 non-null  object             
 6   block_number           100000 non-null  Int64              
 7   block_timestamp        100000 non-null  datetime64[us, UTC]
 8   block_timestamp_month  100000 non-null  dbdate             
 9   input_count            100000 non-null  Int64              
 10  output_count           100000 non-null  Int64              
 11  input_value            99966 non-null   

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,1807841875a802a3d02ece2b1eca412426ac69233d2b7c...,290,263,1,0,00000000000000000002b63ce4c3915ffa104758a2c706...,623708,2020-03-31 04:00:16+00:00,2020-03-01,0,3,None,1253801761.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
1,0d2597f7a973c2bda367b1357b985a3a23f6dd82b642bd...,291,264,2,0,000000000000000000048f5aa966c579bfce03dddee82f...,620192,2020-03-04 19:43:52+00:00,2020-03-01,0,3,None,1263769854.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_HASH160 f870b7..."
2,1f2279feb9f63ecdf5180f00c91b7473283cf1f41f2558...,295,268,2,0,00000000000000000007d634a1052c87f2b40f5c9c0289...,620854,2020-03-08 22:23:24+00:00,2020-03-01,0,3,None,1253662783.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_HASH160 804afd..."
3,671d4dc5d3f9ee15260ceceba7899567535e09efd0da9e...,300,273,1,0,000000000000000000007e499b0c4618b3c3322331229a...,622754,2020-03-24 09:27:14+00:00,2020-03-01,0,3,None,1261885807.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
4,65d1cdc2261936f4ab4fa8710482d08a9a9ff0371ea3b6...,300,273,1,0,00000000000000000005e5ce8fccfd628117703cd3f841...,623392,2020-03-29 02:02:21+00:00,2020-03-01,0,3,None,1266736175.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
5,0d341688fce422327455104d42cab4673613d35960b4b7...,300,273,1,0,0000000000000000000b676f2eecde60357e15e003b51f...,620816,2020-03-08 15:17:10+00:00,2020-03-01,0,3,None,1268917534.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
6,9f88f467cb6aa7c0cdfb8adbe6ed7c30add41f6d606528...,377,350,1,1026257737,000000000000000000047084cad466b17be99f04dd7bca...,622976,2020-03-26 07:51:27+00:00,2020-03-01,0,4,None,1313383125.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
7,03de8288bb8b1225402003c16fff24c0bc0e108cb28a65...,377,350,1,1034436705,000000000000000000015446d101bac1fde0a1408adee6...,622693,2020-03-23 22:03:14+00:00,2020-03-01,0,4,None,1301088826.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
8,6a5ea2df30bf48910deecc1259931971b29cfc2248b94a...,377,350,1,1016828764,0000000000000000000346eec19720bd22137b54719dbe...,619897,2020-03-02 22:35:55+00:00,2020-03-01,0,4,None,1264605335.000000000,True,0E-9,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
9,258ed361e7844ea0ce292d9b3155c047d8bbd54d11dc23...,343,262,1,0,00000000000000000009092d290dd9495c1c08d89de9a9...,623056,2020-03-26 17:58:07+00:00,2020-03-01,1,5,1598870.000000000,1577211.000000000,False,21659.000000000,"[{'index': 0, 'spent_transaction_hash': '189a8...","[{'index': 0, 'script_asm': 'OP_HASH160 8f6cdc..."


In [8]:
(df_transactions['block_timestamp'].min(), df_transactions['block_timestamp'].max())

(Timestamp('2020-03-01 00:04:03+0000', tz='UTC'),
 Timestamp('2020-03-31 23:51:06+0000', tz='UTC'))

In [9]:
# doing a small check to see if I can join things on the blocks and transactions table:
query_join_test = """
SELECT
  t.hash AS transaction_id,
  t.block_number,
  b.hash AS block_hash,
  b.timestamp AS block_time,
  t.fee,
  t.input_value,
  t.output_value
FROM
  `bigquery-public-data.crypto_bitcoin.transactions` t
JOIN
  `bigquery-public-data.crypto_bitcoin.blocks` b
ON
  t.block_number = b.number
WHERE
  t.block_timestamp BETWEEN '2020-01-01' AND '2020-06-30'
LIMIT 10
"""
df_join_test = client.query(query_join_test).to_dataframe()
df_join_test.head()

,transaction_id,block_number,block_hash,block_time,fee,input_value,output_value
0,a25aeee1ab0e62e390c67ab74e275c454ac517f0407e01...,623188,0000000000000000000c4442bab4c000caed889421dd76...,2020-03-27 13:41:00+00:00,0E-9,None,1309076561.000000000
1,dcc79d2db529e1b54cf8e00748a997db9ed79c90e3206c...,622186,000000000000000000090743c7ca9ebc0f825b6534b257...,2020-03-19 11:47:28+00:00,0E-9,None,1338316144.000000000
2,2a6a36c9c38c77d2e94a0d1845593cd5c2de62ac1f28da...,623193,00000000000000000012cb92f8e92fc9891b48a83e7b86...,2020-03-27 14:50:19+00:00,0E-9,None,1299735368.000000000
3,edbcb6893cc0b8e9708a415c63b1dbe134bb42a3aad721...,623294,00000000000000000012cc8d60b7a7e3b7ae9f42b3fbfe...,2020-03-28 06:23:34+00:00,0E-9,None,1250053273.000000000
4,5d31a1410788db81951338e83bfbc439f32729347e7a8b...,621829,00000000000000000003617efd8d2e710ae44626775845...,2020-03-16 08:56:54+00:00,0E-9,None,1338441199.000000000


In [10]:
# SQLite start
import sqlite3


In [11]:
# Connecting to a new SQLite database to create a file for my dataset
connection = sqlite3.connect("bitcoin_data.db")

In [14]:
# Saving the blocks DataFrame into the bitcoin_data.db database as a new table called "blocks"
table_name = "blocks"
data = df_blocks
# Writing the data in df_blocks to the database
data.to_sql(
        name = table_name,
        con = connection,
        if_exists = "replace",
        index = False
)


NameError: name 'df_blocks' is not defined

In [ ]:
# Feature engineering 1 - Converting block timestamp row to extract an hour within the day time column
# This is to extrat the transaction hour out of the data (100K rows of transactions that took place during March 2020) I have
df_transactions['tx_hour'] = df_transactions['block_timestamp'].dt.hour

In [ ]:
# Feature engineering 2 - extracting the fee_rate of the individual transactions that took place out of the 100K rows of transactions that were isolated for the dataset
# This is to calculate the efficiency (how much the sender spent in total BTC amount to send the intended amount of BTC to the recipient): This extracts the "fee rate" or the % amount of BTC paid to the miner in that transaction
# df_transactions['fee_rate'] = df_transactions['fee'] / df_transactions['input_value']
# Since there were small number of rows that returned the value "None" in the "input_value" I modified my code to avoid dividing by 0 or "None"
def calculate_fee_rate(row):
    fee = row['fee']
    input_value = row['input_value']
    
    if input_value in [0, None]:  # To avoid divide by 0 or None
        return None
    else:
        return fee / input_value

df_transactions['fee_rate'] = df_transactions.apply(calculate_fee_rate, axis=1)

In [22]:
# Feature engineering 3 - extracting the value difference the individual transactions that took place out of the 100K rows of transactions that were isolated for the dataset
# This is to calculate the difference of how much the sender spent in total BTC amount to send the intended amount of BTC to the recipient to isolate the "fee" or the amount of BTC paid to the miner in that transaction
# df_transactions['value_diff'] = df_transactions['output_value'] - df_transactions['input_value']
# Since there were small number of rows that returned the value "None" in the "input_value" I modified my code to avoid calculation that uses "None"
def calculate_value_diff(row):
    input_val = row['input_value']
    output_val = row['output_value']
    
    if input_val is None or output_val is None:
        return None
    else:
        return output_val - input_val

df_transactions['value_diff'] = df_transactions.apply(calculate_value_diff, axis=1)

In [23]:
df_transactions[['input_value', 'output_value', 'value_diff']].head(10)

,input_value,output_value,value_diff
0,None,1253801761.000000000,None
1,None,1263769854.000000000,None
2,None,1253662783.000000000,None
3,None,1261885807.000000000,None
4,None,1266736175.000000000,None
5,None,1268917534.000000000,None
6,None,1313383125.000000000,None
7,None,1301088826.000000000,None
8,None,1264605335.000000000,None
9,1598870.000000000,1577211.000000000,-21659.000000000
